In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline


import pandas as pd
import requests
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import codecs, json


In [ ]:
with codecs.open('/content/mlflashcards_tweets_large.json', 'r', 'utf-8') as f:
    tweets = json.load(f, encoding='utf-8')

In [ ]:
len(tweets)


241

In [ ]:
t = tweets[5]
t

{'fullname': 'Chris Albon',
 'id': '960977759915851776',
 'likes': '6',
 'replies': '2',
 'retweets': '1',
 'text': 'Alpha In Ridge Regression https://machinelearningflashcards.com\xa0pic.twitter.com/DFdSKO7DiH',
 'timestamp': '2018-02-06T20:45:26',
 'url': '/chrisalbon/status/960977759915851776',
 'user': 'chrisalbon'}

In [ ]:
def get_tweet_url(tweet):
    return 'https://twitter.com' + tweet['url']

In [ ]:
tweet_url = get_tweet_url(t)
print(tweet_url)

https://twitter.com/chrisalbon/status/960977759915851776


In [ ]:
def get_tweet_text(tweet):
    text = tweet['text']
    res = re.search('(.*) https.*', text)
    if res:
        text = res.group(1)
    else:
        text = None
    return text


In [ ]:
get_tweet_text(t)


'Alpha In Ridge Regression'

In [ ]:
rows = []
for tweet in tweets[:]:
    row = {"id": tweet['id'],
            "likes": tweet['likes'],
            "replies": tweet['replies'],
            "retweets": tweet['retweets'],
            "timestamp": tweet['timestamp'],
            "url": get_tweet_url(tweet),
            "text": get_tweet_text(tweet)}
    rows.append(row)

In [ ]:
print(len(rows))
#rows

241


In [ ]:
df = pd.DataFrame.from_dict(rows)
df.head()


,id,likes,replies,retweets,timestamp,url,text
0,892802102702911488,1,0,1,2017-08-02T17:39:43,https://twitter.com/chrisalbon/status/89280210...,None
1,961698946698567680,5,0,0,2018-02-08T20:31:11,https://twitter.com/chrisalbon/status/96169894...,Threshold Activation
2,961666291189743616,23,0,5,2018-02-08T18:21:25,https://twitter.com/chrisalbon/status/96166629...,Chi-Squared
3,961337263299821568,28,1,4,2018-02-07T20:33:59,https://twitter.com/chrisalbon/status/96133726...,MinMax Scaling
4,961302907969654784,22,0,2,2018-02-07T18:17:28,https://twitter.com/chrisalbon/status/96130290...,Probability Density Function


In [ ]:
def get_img_url(tweet_url):
    page_data = requests.get(tweet_url).text
    res = re.search('data-image-url="(.*)"', page_data)
    if res:
        img_url = res.group(1)
    else:
        img_url = None
    return img_url

In [ ]:
get_img_url(tweet_url)
df['img_url'] = [get_img_url(tweet_url)  for tweet_url in df.url]
df.tail()

In [ ]:
df.to_csv("chrisalbon_mlflashcards.csv", index=False)
img_url = 'https://pbs.twimg.com/media/DGPcXPAVwAAfbqV.jpg'
img_data = requests.get(img_url).content
with open('test.png', 'wb') as handler:
    handler.write(img_data)
from IPython.display import Image
Image("test.png")

In [ ]:

import requests
import tweepy
import re
import os

In [ ]:
auth = tweepy.OAuthHandler('3nVuSoBZnx6U4vzUxf5w','Bcs59EFbbsdF6Sl9Ng71smgStWEGwXXKSjYvPVt7qys')
# Construct the API instance
api = tweepy.API(auth)

# query parameter excludes the intentional photo to dirty the data
params = {'q': '-"are%20-being%20-made"%20-"your%20-data%20-dirty"%20%23machinelearningflashcards-filter:retweets',
          'from': 'chrisalbon',
          'since': '2017-04-01', }

In [ ]:
ml_flashcards_json = api.search(**params)
ml_flashcards_json

[]

In [ ]:
media_urls = []
titles = []
for i in range(len(ml_flashcards_json)):
        txt = ml_flashcards_json[i]
        json = txt._json
        title = re.sub("#\S*", "", str(json['text'])) # removes hashtag
        title = re.sub("http\S*", "", title) # removes url
        title = str(title.strip()) # strips spacing away
        try: # KeyError is raised when there is no image attached to the tweet
            media_url = json['entities']['media'][0]['media_url']
            media_urls.append(media_url)
            titles.append(title)
        except KeyError:
            counter = 0
            counter += 1
            print("{} tweet was not processed due to a KeyError".format(counter))

In [ ]:
current_dirctory = os.getcwd()
os.chdir(current_dirctory)
for i in zip(media_urls, titles):
    print("1")
    img = requests.get(i[0])
    f = open(str(i[1]) + ".jpg", mode='wb')
    f.write(img.content)
    f.close()